In [1]:
import sys
from pathlib import Path
import numpy as np
from matplotlib import pyplot as plt
from pprint import pprint
import json
from tabulate import tabulate

ROOT = Path().absolute()
DATASETS_PATH = ROOT / "hloc/pipelines/Blender_Synthetic/datasets"
SFM_PATH = ROOT / "hloc/pipelines/Blender_Synthetic/out"

if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

In [2]:
# read stats for datasets
datasets_stats = []
for p in DATASETS_PATH.rglob("stats.json"):
    ds_name = p.parent.name
    #print(f"  {ds_name}")
    with open(p, "r", encoding="utf8") as f:
        data = json.load(f)
    datasets_stats.append((data, ds_name))

print(f"found {len(datasets_stats)} dataset stats")
# pprint(datasets_stats)

# read stats for sfm results
sfm_stats = []
for p in SFM_PATH.rglob("stats.json"):
    ds_name = p.parent.parent.parent.name
    sfm_method = p.parent.name.removeprefix(ds_name).removeprefix("-")
    #print(f"  {ds_name} - {sfm_method}")
    with open(p, "r", encoding="utf8") as f:
        data = json.load(f)
    sfm_stats.append((data, ds_name, sfm_method))

print(f"found {len(sfm_stats)} sfm stats")
# pprint(sfm_stats)

found 12 dataset stats
found 50 sfm stats


In [3]:
PLACEHOLDER = "-"

# analyse ds stats
table = []
for data, ds_name in datasets_stats:
    c2c_stats = [x for x in data["stats"] if x["full_name"] == "Average Camera to Camera Centroid Distance"]
    c2c_stats = c2c_stats[0] if len(c2c_stats) > 0 else None
    baseline_stats = [x for x in data["stats"] if x["full_name"] == "Min and Max View Baseline"]
    baseline_stats = baseline_stats[0] if len(baseline_stats) > 0 else None
    table.append(
        [
            f'{data["scene_name"]} ({data["env_background"]})',
            data["num_cameras"],
            PLACEHOLDER if not c2c_stats else f'{c2c_stats["value"]:.2f}',
            PLACEHOLDER if not baseline_stats else f'{baseline_stats["min"]:.2f}',
            PLACEHOLDER if not baseline_stats else f'{baseline_stats["max"]:.2f}',
        ]
    )

print(
    tabulate(
        table,
        [
            "name",
            "cams",
            "avg. c2c centroid dist (m)",
            "min view dist (m)",
            "max view dist (m)",
        ],
        tablefmt="plain",
        showindex="always",
    )
)

    name                              cams    avg. c2c centroid dist (m)    min view dist (m)    max view dist (m)
 0  Bartholomew (evening_field_8k)      10                          4.39                 2.43                14.42
 1  Chateu (evening_field_8k)           15                         11.26                 2.71                32.2
 2  Chateu (evening_field_8k)           15                         11.26                 2.71                32.2
 3  Chateu (evening_field_8k)           15                         11.26                 2.71                32.2
 4  Framlingham (evening_field_8k)      10                         11.07                 4.22                39.68
 5  Bartholomew (evening_field_8k)      10                          4.39                 2.43                14.42
 6  Chateu (evening_field_8k)           15                         11.26                 2.71                32.2
 7  Chateu (evening_field_8k)           15                         11.26            

In [4]:
# analyse ds stats
table = []
for data, ds_name, sfm_method in sfm_stats:
    if "result" in data and data["result"] == "error":
        table.append(
            [
                ds_name,
                f"{sfm_method} *",
                PLACEHOLDER,
                PLACEHOLDER,
                PLACEHOLDER,
                PLACEHOLDER,
                PLACEHOLDER,
                PLACEHOLDER,
                PLACEHOLDER,
            ]
        )
        continue
    t_errs = np.array([i["t_err_m"] for i in data["analysed_poses"]])
    r_errs = np.array([i["r_err_deg"] for i in data["analysed_poses"]])
    re_errs = np.array([i["reprojection_err"] for i in data["analysed_poses"]])
    t_errs_med = np.median(t_errs)
    r_errs_med = np.median(r_errs)
    loc = [i for i in data["analysed_poses"] if "MAIN" in i["name"]][0]
    loc_t_err = loc["t_err_m"]
    loc_r_err = loc["r_err_deg"]
    re_errs_med = np.median(re_errs)
    loc_abs_ground_distance = loc["ground_distance_m"]

    # ratios
    t_errs_size = t_errs.size
    r_errs_size = r_errs.size
    t_ratio = t_errs[t_errs < 1].size / t_errs_size
    r_ratio = r_errs[r_errs < 5].size / r_errs_size

    table.append(
        [
            ds_name,
            sfm_method,
            f"{re_errs_med:.2f}",
            f"{t_errs_med:.2f}",
            f"{r_errs_med:.2f}",
            f"{loc_t_err:.2f}",
            f"{loc_r_err:.2f}",
            f"{loc_abs_ground_distance:.2f}",
            #f"{t_ratio:.2f}",
            #f"{r_ratio:.2f}",
        ]
    )

print(
    tabulate(
        table,
        [
            "dataset",
            "feature pipeline",
            "med. reprojection error (px)",  # median, because main cam is outlier
            "med. t_err (m)",
            "med. r_err (°)",
            "localized camera t_err (m)",
            "localized camera r_err (°)",
            "distance to ground (m)"
            #"t_err (x < 1px)",
            #"r_err (x < 5°)",
        ],
        tablefmt="plain",
        showindex="always",
    ),
    "\n\n* this method failed to localize the camera",
)

    dataset                           feature pipeline        med. reprojection error (px)    med. t_err (m)    med. r_err (°)    localized camera t_err (m)    localized camera r_err (°)    distance to ground (m)
 0  Bartholomew+evening_field_8k      disk+lightglue          0.87                            0.03              0.21              1.29                          2.39                          1.11
 1  Bartholomew+evening_field_8k      loftr                   0.60                            0.01              0.05              1.58                          2.17                          0.88
 2  Bartholomew+evening_field_8k      superpoint+lightglue    1.12                            0.02              0.21              4.97                          1.62                          0.97
 3  Bartholomew+evening_field_8k      superpoint+superglue    1.11                            0.02              0.18              4.12                          1.64                          1.00
 4  Cha